<a href="https://colab.research.google.com/github/ikechukwuUE/steel-plate-defect/blob/master/steel_plate_prediction_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enhancing Steel Plate Defect Prediction with Advanced Feature Engineering and Neural Networks

## Table of Contents

1. **Introduction**
2. **Data Preparation**
3. **Model Construction**
    - **Traditional Machine Learning Models**
    - **Neural Networks**
4. **Ensemble and Tuning**
5. **Execution**
6. **Conclusion**
7. **Appendices**


## Introduction

### Project Overview
- **Objective:** Develop a sophisticated machine learning model to predict the probability of various defects on steel plates using both the competition dataset and the original Steel Plates Faults dataset from UCI.
- **Methodology:** Focus on extensive feature engineering using Principal Component Analysis (PCA) to reduce the dimensionality of the dataset and incorporate neural networks for defect prediction.
- **Expected Outcome:** A CSV file with predicted probabilities for each defect category for each id in the test set, evaluated using the area under the ROC curve (AUC) for each category.

### Version Details
- **Version Number:** 1.0
- **Configuration Parameters:** Detailed in the Configuration Parameters section.

## Imports

In [ ]:
## Imports

%%time

# Installing select libraries
!pip install -q lightgbm==4.3.0 --force-reinstall
!pip install --force-reinstall scikit-learn --no-index --find-links=file:///kaggle/input/scikit-learn-1-4-0/

# General library imports
from gc import collect
from warnings import filterwarnings
filterwarnings('ignore')
from IPython.display import display_html, clear_output
clear_output()

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import sklearn as sk
import pandas as pd
print(f"---> XGBoost = {xgb.__version__} | LightGBM = {lgb.__version__} | Catboost = {cb.__version__}")
print(f"---> Sklearn = {sk.__version__}| Pandas = {pd.__version__}\n\n")
collect()

# Data manipulation and visualization
from copy import deepcopy
import numpy as np
import re
from scipy.stats import mode, kstest, normaltest, shapiro, anderson, jarque_bera
from collections import Counter
from itertools import product
from colorama import Fore, Style, init
init(autoreset=True)
import joblib
import os

from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap as LCM
%matplotlib inline

from pprint import pprint
from functools import partial

# Model and pipeline specifics
from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, FunctionTransformer as FT, PowerTransformer
from sklearn.impute import SimpleImputer as SI
from sklearn.model_selection import RepeatedStratifiedKFold as RSKF, StratifiedKFold as SKF, StratifiedGroupKFold as SGKF, KFold, RepeatedKFold as RKF, cross_val_score, cross_val_predict
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

# ML Model training
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer
from xgboost import DMatrix, XGBClassifier as XGBC
from lightgbm import log_evaluation, early_stopping, LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC, Pool
from sklearn.ensemble import HistGradientBoostingClassifier as HGBC, RandomForestClassifier as RFC

# Ensemble and tuning
import optuna
from optuna import Trial, trial, create_study
from optuna.pruners import HyperbandPruner
from optuna.samplers import TPESampler, CmaEsSampler
optuna.logging.set_verbosity = optuna.logging.ERROR

In [ ]:
# Setting rc parameters in seaborn for plots and graphs
sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.75,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : '#0099e6',
         'axes.titlesize'       : 8.5,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        })


In [ ]:
# Color printing
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string"
    print(style + color + text + Style.RESET_ALL)

# Making sklearn pipeline outputs as dataframe
from sklearn import set_config
set_config(transform_output = "pandas")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

print()
collect()


In [ ]:
# Function to load and preprocess data
def load_and_preprocess_data(train_path, test_path):
    # Load datasets
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # Preprocessing steps (e.g., handling missing values, encoding categorical variables)
    # Example: train_data = train_data.fillna(train_data.mean())
    # Example: test_data = test_data.fillna(test_data.mean())

    return train_data, test_data

# Function to split data into features and target
def split_data(data, target_column):
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    return X, y

# Function to apply PCA
def apply_pca(X, n_components):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca

# Function to train and evaluate a model
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Function to plot ROC curve
def plot_roc_curve(y_test, y_pred_proba):
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

# Example usage
# train_data, test_data = load_and_preprocess_data('path/to/train_data.csv', 'path/to/test_data.csv')
# X_train, y_train = split_data(train_data, 'target_column')
# X_test, y_test = split_data(test_data, 'target_column')
# X_train_pca = apply_pca(X_train, n_components=10)
# X_test_pca = apply_pca(X_test, n_components=10)
# model = RandomForestClassifier()
# accuracy = train_and_evaluate_model(model, X_train_pca, y_train, X_test_pca, y_test)
# print(f"Accuracy: {accuracy}")
# y_pred_proba = model.predict_proba(X_test_pca)
# plot_roc_curve(y_test, y_pred_proba)